<a href="https://colab.research.google.com/github/meike23/DSMeetup/blob/master/_Data_Science_Fundamentals_The_Pandas_Library_Box_Office_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Fundamentals: The Pandas Library
## Box Office Data

Today, we will be exploring fundamental concepts of pandas data manipulation to prepare a data set for modeling. 

__Data set:__ Information from IMDB about movies

__Our Goal:__ Process and clean the data to prepare it for modeling to predict the gross profit of a movie.

You will need to add some code to complete this notebook.  Follow along with the instructor to find what code to add.  You will add that where the code says "\*\*\* ADD CODE HERE\*\*\*"

Have fun and good luck coding!

## Importing the packages that we'll need

One of the things that makes Python **great** for data science is all of the different libraries that exist so we don't have to code them from scratch. Tonight we'll be taking advantage of:
- [Numpy](https://numpy.org/) for scientific and mathematical computing
- [Pandas](https://pandas.pydata.org/) for data wrangling and analysis

In [0]:
# data analysis packages
import numpy as np
import pandas as pd

# default settings for pandas
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

## Import the data
Pandas can work with information from all kinds of data sources. Below, we'll import the data we need from a GitHub URL and read it into a Pandas Dataframe using the Pandas [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function.

In [0]:
# import data from github
data = pd.read_csv("https://github.com/autumntoney/predict_the_box_office/raw/master/movie_metadata.csv")

## Understand the data

In [9]:
# Check out the first lines of the data set
data.head(6)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.00,178.00,0.00,855.00,Joel David Moore,"1,000.00","760,505,847.00",Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.00,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,"3,054.00",English,USA,PG-13,"237,000,000.00","2,009.00",936.00,7.90,1.78,33000
1,Color,Gore Verbinski,302.00,169.00,563.00,"1,000.00",Orlando Bloom,"40,000.00","309,404,152.00",Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.00,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,"1,238.00",English,USA,PG-13,"300,000,000.00","2,007.00","5,000.00",7.10,2.35,0
2,Color,Sam Mendes,602.00,148.00,0.00,161.00,Rory Kinnear,"11,000.00","200,074,175.00",Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.00,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.00,English,UK,PG-13,"245,000,000.00","2,015.00",393.00,6.80,2.35,85000
3,Color,Christopher Nolan,813.00,164.00,"22,000.00","23,000.00",Christian Bale,"27,000.00","448,130,642.00",Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.00,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,"2,701.00",English,USA,PG-13,"250,000,000.00","2,012.00","23,000.00",8.50,2.35,164000
4,NaN,Doug Walker,nan,nan,131.00,nan,Rob Walker,131.00,nan,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.00,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,nan,NaN,NaN,NaN,nan,nan,12.00,7.10,nan,0
5,Color,Andrew Stanton,462.00,132.00,475.00,530.00,Samantha Morton,640.00,"73,058,679.00",Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,212204,1873,Polly Walker,1.00,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,738.00,English,USA,PG-13,"263,700,000.00","2,012.00",632.00,6.60,2.35,24000


In [10]:
# Checking the size of our data (rows, columns)
data.shape

(5043, 28)

In [13]:
# Get a concise summary of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   director_name              4939 non-null   object 
 2   num_critic_for_reviews     4993 non-null   float64
 3   duration                   5028 non-null   float64
 4   director_facebook_likes    4939 non-null   float64
 5   actor_3_facebook_likes     5020 non-null   float64
 6   actor_2_name               5030 non-null   object 
 7   actor_1_facebook_likes     5036 non-null   float64
 8   gross                      4159 non-null   float64
 9   genres                     5043 non-null   object 
 10  actor_1_name               5036 non-null   object 
 11  movie_title                5043 non-null   object 
 12  num_voted_users            5043 non-null   int64  
 13  cast_total_facebook_likes  5043 non-null   int64

In [14]:
# Understand the basic statistical details of the data set
data.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,"4,993.00","5,028.00","4,939.00","5,020.00","5,036.00","4,159.00","5,043.00","5,043.00","5,030.00","5,022.00","4,551.00","4,935.00","5,030.00","5,043.00","4,714.00","5,043.00"
mean,140.19,107.20,686.51,645.01,"6,560.05","48,507,385.63","83,668.16","9,699.06",1.37,272.77,"39,752,620.44","2,002.47","1,651.75",6.44,2.22,"7,525.96"
std,121.60,25.20,"2,813.33","1,665.04","15,020.76","68,471,915.43","138,485.26","18,163.80",2.01,377.98,"206,114,898.45",12.47,"4,042.44",1.13,1.39,"19,320.45"
min,1.00,7.00,0.00,0.00,0.00,162.00,5.00,0.00,0.00,1.00,218.00,"1,916.00",0.00,1.60,1.18,0.00
25%,50.00,93.00,7.00,133.00,614.00,"5,351,178.00","8,593.50","1,411.00",0.00,65.00,"6,000,000.00","1,999.00",281.00,5.80,1.85,0.00
50%,110.00,103.00,49.00,371.50,988.00,"25,528,495.00","34,359.00","3,090.00",1.00,156.00,"20,000,000.00","2,005.00",595.00,6.60,2.35,166.00
75%,195.00,118.00,194.50,636.00,"11,000.00","62,319,957.00","96,309.00","13,756.50",2.00,326.00,"45,000,000.00","2,011.00",918.00,7.20,2.35,"3,000.00"
max,813.00,511.00,"23,000.00","23,000.00","640,000.00","760,505,847.00","1,689,764.00","656,730.00",43.00,"5,060.00","12,215,500,000.00","2,016.00","137,000.00",9.50,16.00,"349,000.00"


__Missing Values:__ You can identify columns with missing values by looking at the `non-null` items in the `.info()` call, and the difference in number of data points in the `count` row of the `.describe()` function. For example, the `num_critic_for_reviews` column has a lower `count` than `duration`, so has missing values.

__Outliers:__ You can also identify if there are outliers in the data set by looking at the `.describe()` function's spread of data in comparison to the `min` and `max` values.  For example, the `duration` column's `max` value is 511, which would correspond to an 8.5 hour film. You might want to go back and check that the values for that film are valid, and correct or drop the values that aren't.  

NB: We don't have time to investigate outliers in this workshop.

## Clean the Data

Now that we understand the basics of what's in the data, we now need to clean the data before it's ready for modeling.  Things we'll cover:
- Duplicate data
- Missing data
- Manipulating data

NB: Most models can only process numerical data, so we will focus our cleaning on those columns.

In [17]:
# Rename columns to include units
data.rename(columns={'duration': 'duration_mins',
                    'budget': 'budget_usd',
                    'gross': 'gross_usd'}, inplace=True)
data.head(1)

,color,director_name,num_critic_for_reviews,duration_mins,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross_usd,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget_usd,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.00,178.00,0.00,855.00,Joel David Moore,"1,000.00","760,505,847.00",Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.00,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,"3,054.00",English,USA,PG-13,"237,000,000.00","2,009.00",936.00,7.90,1.78,33000


### Removing duplicates
Since our data includes that of movies, we will want to check for duplicated `movie_title`, but since there could be movie remakes in here as well, we should also check the `title_year` using the Pandas [`duplicated`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function.

In [18]:
# Checking to see what movies are duplicates.  
# Sorting by movie title to see duplicates
data[data.duplicated(subset=['movie_title', 'title_year'], keep=False)].sort_values('movie_title').head()

,color,director_name,num_critic_for_reviews,duration_mins,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross_usd,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget_usd,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
3711,Color,Richard Fleischer,69.00,127.00,130.00,51.00,Robert J. Wilke,617.00,nan,Adventure|Drama|Family|Fantasy|Sci-Fi,James Mason,"20,000 Leagues Under the Sea",22123,799,Paul Lukas,0.00,captain|expedition|sea|submarine|whale,http://www.imdb.com/title/tt0046672/?ref_=fn_t...,108.00,English,USA,Approved,"5,000,000.00","1,954.00",53.00,7.20,1.37,0
4894,Color,Richard Fleischer,69.00,127.00,130.00,51.00,Robert J. Wilke,618.00,nan,Adventure|Drama|Family|Fantasy|Sci-Fi,James Mason,"20,000 Leagues Under the Sea",22124,800,Paul Lukas,0.00,captain|expedition|sea|submarine|whale,http://www.imdb.com/title/tt0046672/?ref_=fn_t...,108.00,English,USA,Approved,"5,000,000.00","1,954.00",53.00,7.20,1.37,0
4950,Color,David Hewlett,8.00,88.00,686.00,405.00,David Hewlett,847.00,nan,Comedy,Christopher Judge,A Dog's Breakfast,3262,2364,Paul McGillion,2.00,dog|vegetarian,http://www.imdb.com/title/tt0796314/?ref_=fn_t...,46.00,English,Canada,NaN,"120,000.00","2,007.00",686.00,7.00,1.78,377
4949,Color,David Hewlett,8.00,88.00,686.00,405.00,David Hewlett,847.00,nan,Comedy,Christopher Judge,A Dog's Breakfast,3262,2364,Paul McGillion,2.00,dog|vegetarian,http://www.imdb.com/title/tt0796314/?ref_=fn_t...,46.00,English,Canada,NaN,"120,000.00","2,007.00",686.00,7.00,1.78,377
1420,Color,Wes Craven,256.00,101.00,0.00,574.00,Lin Shaye,"40,000.00","26,505,000.00",Horror,Johnny Depp,A Nightmare on Elm Street,149285,42918,Amanda Wyss,1.00,freddy krueger|lucid dream|serial killer|slash...,http://www.imdb.com/title/tt0087800/?ref_=fn_t...,668.00,English,USA,X,"1,800,000.00","1,984.00",852.00,7.50,1.85,10000


We do have duplicated movies in our data.  We need to drop those duplicates to remove them from our data set using the Pandas [`drop_duplicates`](https://pandas.pydata.org/pandas-docs/version/0.17/generated/pandas.DataFrame.drop_duplicates.html) function.

In [0]:
# Drop all duplicate movie titles that were released in the same year
data = data.drop_duplicates(subset=['movie_title', 'title_year'], keep='first').copy()

### Missing values
Now that we've cleaned out the duplicates, let's take a look at the missing values.

In [23]:
# Show how many values are missing from each column
data.isna().head()

,color,director_name,num_critic_for_reviews,duration_mins,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross_usd,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget_usd,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,True,True,False,True,False,False,True,False,False,False,False,False,True,False,True,False,True,True,True,True,True,True,False,False,False


There are a good number of films missing the `aspect_ratio`, and using our knowledge of movies, it is unlikely that the aspect ratio is important in predicting the gross revenue of the film.  Let's drop the whole column using the Pandas [`drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) function.

In [54]:
# Drop the aspect ratio column,  axis=1 means drop the column
data.drop('aspect_ratio', axis=1, inplace=True)

KeyError: ignored

Now, since we're wanting to eventually predict the `gross` revenue of the movie, let's take a look at the missing values of `gross`

In [20]:
# What percent of the gross values are missing?
print(len(data[data['gross_usd'].isna()])/len(data))
# Or, more simply
data['gross_usd'].isna().mean()

0.17544216304126856


0.17544216304126856

In [27]:
# show what movies are missing the gross values
data[data['gross_usd'].isna()].head()

,color,director_name,num_critic_for_reviews,duration_mins,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross_usd,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget_usd,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes


Since we eventually want to predict the gross revenue of a movie, having a null value for gross revenue will not help us train our model, so we should drop them using the Pandas [`dropna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) function.

In [25]:
# Drop all null values of gross
# You might be interested in how the movie performed
# so don't want to impute these values (skew the analysis)
data.dropna(subset=['gross_usd'], how='all', inplace=True)
# Check the size of the data after dropping null values
data.shape

(4056, 27)

Check out the highest-grossing movies by sorting the data.

In [28]:
# Sort all of the values by gross
data['gross_usd'].sort_values(ascending=False).head()

0    760,505,847.00
26   658,672,302.00
29   652,177,271.00
17   623,279,547.00
66   533,316,061.00
Name: gross_usd, dtype: float64

Knowing a bit about movies, it makes sense that the budget of a film might be important to predicting how much money the movie makes, so let's check out the missing budget values.

In [29]:
# Identifying the percent of budget values that are missing
data['gross_usd'].sort_values(ascending=False).head()

0    760,505,847.00
26   658,672,302.00
29   652,177,271.00
17   623,279,547.00
66   533,316,061.00
Name: gross_usd, dtype: float64

Since we have between 5-10% of budget values missing, we can impute (or fill in) the missing values without creating too much bias in the data. 

Using our knowledge of films, I know that film budgets in recent years has increased, so just imputing the mean or median budget for all missing values would not make much sense.  To get around this, we can find the median gross revenue for each year using the Pandas [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) function, and then impute based on those values using the [`fillna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) function.

In [0]:
# Drop movies where the year is missing
data.dropna(subset=['title_year'], how='all', inplace=True)
# Convert all years to integers
data['title_year'] = data['title_year'].astype(int)

In [0]:
# Calculate median budgets per year
# Impute the median budgets per year for missing budget data
data['budget_usd'] = data['budget_usd'].fillna(data.groupby('title_year')['budget_usd'].transform('median'))

In [33]:
# Re-check to see if there are still any missing budget values
data['budget_usd'].isna().sum()

1

In [34]:
# What is the movie with missing budget still?
data[data['budget_usd'].isna()].head()

,color,director_name,num_critic_for_reviews,duration_mins,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross_usd,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget_usd,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes
4647,Color,James Algar,116.00,70.00,11.00,8.00,Donnie Dunagan,16.00,"102,797,150.00",Animation|Drama|Family,Sam Edwards,Bambi,94225,63,Ann Gillis,0.00,animal|bambi|deer|forest|hunter,http://www.imdb.com/title/tt0034492/?ref_=fn_t...,136.00,English,USA,Approved,nan,1942,12.00,7.40,0


There are no other movies from the year 1942, so we cannot impute based on year.  Other options:
- Impute based on the overall median budget
- Drop the film from the data
- Impute based on other data

Since it's only one film, and for the sake of time, we will drop the film from the data set. 

In [0]:
data.dropna(subset=['budget_usd'], axis=0, inplace=True)

Now, let's take a look at the different countries of origin in the data.  There are likely differences in the gross revenue of movies from different countries.

In [36]:
# Identify all of the unique countries
data['country'].unique()

array(['USA', 'UK', 'New Zealand', 'Canada', 'Australia', 'Germany',
       'China', 'New Line', 'France', 'Japan', 'Spain', 'Hong Kong',
       'Czech Republic', 'India', 'Peru', 'South Korea', 'Aruba',
       'Denmark', 'Mexico', 'Belgium', 'Ireland', 'South Africa', 'Italy',
       'Romania', 'Chile', 'Netherlands', 'Hungary', 'Russia', 'Greece',
       'Taiwan', 'Official site', 'Thailand', 'Iran', 'West Germany',
       'Georgia', 'Iceland', 'Brazil', 'Finland', 'Norway', 'Sweden',
       'Argentina', 'Colombia', 'Poland', 'Israel', 'Indonesia',
       'Afghanistan', 'Cameroon', 'Philippines'], dtype=object)

In [37]:
# Find how many movies are in each country
counts = data['country'].value_counts()
counts

USA               3153
UK                 352
France             120
Germany             85
Canada              72
Australia           40
Spain               26
India               24
Japan               17
China               17
Hong Kong           15
Mexico              14
Italy               11
South Korea         10
New Zealand         10
Ireland             10
Denmark              9
Thailand             5
Brazil               5
Israel               4
Norway               4
Iran                 4
Russia               4
Netherlands          3
Sweden               3
Argentina            3
South Africa         3
Czech Republic       3
Taiwan               2
Hungary              2
Belgium              2
Romania              2
Poland               2
Iceland              2
Greece               2
Peru                 1
Colombia             1
Aruba                1
West Germany         1
Georgia              1
Finland              1
Cameroon             1
Chile                1
Philippines

Since most movies are from the US, UK, and France let's just focus on movies from those three countries, so the other countries don't skew our predictions.

In [39]:
# Select just the countries with the 3 largest number of films
counts.nlargest(3)

USA       3153
UK         352
France     120
Name: country, dtype: int64

In [41]:
# Select just the country names of the three largest
counts.nlargest(3).index

Index(['USA', 'UK', 'France'], dtype='object')

In [0]:
# Select the data from only the top 3 countries
data = data[data['country'].isin(counts.nlargest(3).index)].copy()

Now, let's take a look at how our cleaning is going.

In [43]:
# Check again to see how many missing values we have
data.isna().sum()

color                         1
director_name                 0
num_critic_for_reviews        3
duration_mins                 1
director_facebook_likes       0
actor_3_facebook_likes        9
actor_2_name                  3
actor_1_facebook_likes        2
gross_usd                     0
genres                        0
actor_1_name                  2
movie_title                   0
num_voted_users               0
cast_total_facebook_likes     0
actor_3_name                  9
facenumber_in_poster          6
plot_keywords                25
movie_imdb_link               0
num_user_for_reviews          1
language                      3
country                       0
content_rating               36
budget_usd                    0
title_year                    0
actor_2_facebook_likes        3
imdb_score                    0
movie_facebook_likes          0
dtype: int64

We have very few missing values remaining -- if we had more time, we could address each individually, but we don't.  Let's drop all remaining rows that have missing values.

In [46]:
print(data.shape)
# Dropping all remaining rows that have null values
data.dropna(axis=0, inplace=True)
print(data.shape)

(3625, 27)
(3555, 27)


## Feature Engineering
Sometimes you might want to use your knowledge of the subject to create new features to help you make predictions

Here, I think that the lead actor might have an influence on the gross revenue of a film, so let's check those out to see if we can turn these into a numerical feature for modeling.

In [47]:
# Check out the actors with the most movies in the set
data['actor_1_name'].value_counts().head()

Robert De Niro       41
Johnny Depp          34
J.K. Simmons         29
Denzel Washington    29
Matt Damon           28
Name: actor_1_name, dtype: int64

The actors with the most lead roles have 20+ lead roles in this data set.  Let's make a feature that includes whether or not the lead actor has starred in a lot of movies (20+).

For this, we will use a list comprehension, which is a concise way to create a list.  The basic syntax for this is:
- `[expression if conditional for item in list]`
  - where the `expression` is based on an item in the list, 
  - `if conditional` filters down the list
  - `for item in list` breaks down a list into individual items. 
 

In [48]:
# List comprehension example
example_list = [1, 2, 3, 4]
[number for number in example_list if number % 2 == 0]

[2, 4]

In [49]:
# Identify all movie counts, select all star actors
lead_movie_counts = data['actor_1_name'].value_counts()
star_actors = lead_movie_counts[lead_movie_counts>=20].index
# Set `lead_star` = 1 if actor is in star_actors, otherwise 0
data['lead_star'] = [1 if x in star_actors else 0 for x in data['actor_1_name']]
data.head()

,color,director_name,num_critic_for_reviews,duration_mins,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross_usd,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget_usd,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes,lead_star
0,Color,James Cameron,723.00,178.00,0.00,855.00,Joel David Moore,"1,000.00","760,505,847.00",Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.00,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,"3,054.00",English,USA,PG-13,"237,000,000.00",2009,936.00,7.90,33000,0
1,Color,Gore Verbinski,302.00,169.00,563.00,"1,000.00",Orlando Bloom,"40,000.00","309,404,152.00",Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.00,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,"1,238.00",English,USA,PG-13,"300,000,000.00",2007,"5,000.00",7.10,0,1
2,Color,Sam Mendes,602.00,148.00,0.00,161.00,Rory Kinnear,"11,000.00","200,074,175.00",Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.00,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.00,English,UK,PG-13,"245,000,000.00",2015,393.00,6.80,85000,0
3,Color,Christopher Nolan,813.00,164.00,"22,000.00","23,000.00",Christian Bale,"27,000.00","448,130,642.00",Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.00,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,"2,701.00",English,USA,PG-13,"250,000,000.00",2012,"23,000.00",8.50,164000,0
5,Color,Andrew Stanton,462.00,132.00,475.00,530.00,Samantha Morton,640.00,"73,058,679.00",Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,212204,1873,Polly Walker,1.00,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,738.00,English,USA,PG-13,"263,700,000.00",2012,632.00,6.60,24000,0


Perhaps the `content_rating` will have an impact on the gross revenue of the movie.  We already have this data, but our models will want the information in numerical form, not in words.  

To do this, we need to encode the ratings as numbers.  We can use the pandas [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) function.

In [50]:
# Encoding ratings as dummy variables
content_ratings = pd.get_dummies(data['content_rating'])
content_ratings.head(2)

,Approved,G,GP,M,NC-17,Not Rated,PG,PG-13,Passed,R,Unrated,X
0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0


In [51]:
# Merge the encoded data back on to the original data
data = data.join(content_ratings)
data.head(3)

,color,director_name,num_critic_for_reviews,duration_mins,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross_usd,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget_usd,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes,lead_star,Approved,G,GP,M,NC-17,Not Rated,PG,PG-13,Passed,R,Unrated,X
0,Color,James Cameron,723.00,178.00,0.00,855.00,Joel David Moore,"1,000.00","760,505,847.00",Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.00,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,"3,054.00",English,USA,PG-13,"237,000,000.00",2009,936.00,7.90,33000,0,0,0,0,0,0,0,0,1,0,0,0,0
1,Color,Gore Verbinski,302.00,169.00,563.00,"1,000.00",Orlando Bloom,"40,000.00","309,404,152.00",Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.00,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,"1,238.00",English,USA,PG-13,"300,000,000.00",2007,"5,000.00",7.10,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,Color,Sam Mendes,602.00,148.00,0.00,161.00,Rory Kinnear,"11,000.00","200,074,175.00",Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.00,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.00,English,UK,PG-13,"245,000,000.00",2015,393.00,6.80,85000,0,0,0,0,0,0,0,0,1,0,0,0,0


Now, we could do a similar type of encoding for any of the other categorical variables, but encoding all of the actor names would create too many columns.  We'll stick with the numerical data for now and select them using the Pandas [`select_dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html) function.

In [53]:
# Select columns by data type - number
numerical_data = data.select_dtypes(include='number')
numerical_data.head()

,num_critic_for_reviews,duration_mins,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross_usd,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget_usd,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes,lead_star,Approved,G,GP,M,NC-17,Not Rated,PG,PG-13,Passed,R,Unrated,X
0,723.00,178.00,0.00,855.00,"1,000.00","760,505,847.00",886204,4834,0.00,"3,054.00","237,000,000.00",2009,936.00,7.90,33000,0,0,0,0,0,0,0,0,1,0,0,0,0
1,302.00,169.00,563.00,"1,000.00","40,000.00","309,404,152.00",471220,48350,0.00,"1,238.00","300,000,000.00",2007,"5,000.00",7.10,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,602.00,148.00,0.00,161.00,"11,000.00","200,074,175.00",275868,11700,1.00,994.00,"245,000,000.00",2015,393.00,6.80,85000,0,0,0,0,0,0,0,0,1,0,0,0,0
3,813.00,164.00,"22,000.00","23,000.00","27,000.00","448,130,642.00",1144337,106759,0.00,"2,701.00","250,000,000.00",2012,"23,000.00",8.50,164000,0,0,0,0,0,0,0,0,1,0,0,0,0
5,462.00,132.00,475.00,530.00,640.00,"73,058,679.00",212204,1873,1.00,738.00,"263,700,000.00",2012,632.00,6.60,24000,0,0,0,0,0,0,0,0,1,0,0,0,0


Now our `numerical_data` DataFrame is ready for modeling!

# Take Home Challenge
- Investigate outliers and clean or drop them
- Analyze your clean data (see Thinkful's Art of Visualization workshops/webinars for additional support)
- Continue to engineer features based on what you know about the subject
  - Maybe the era of the film might have something to do with the gross revenue.  The [history of film](http://www.historyoffilm.net/movie-eras/history-of-cinema/) documents the movie eras we can use to create our new feature along with Pandas [`.cut`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) function.
  - Here is some starter code to get the ball rolling:
  - Identify the era of the film

    era_bins = [0, 1910, 1926, 1940, 1954, 1976, 2000, 2100]

    era_labels = ['pioneer', 'silent', 'talkies', 'golden_era','changes', 'dawn_modern_film', 'modern_film']

    data['era'] = pd.cut(...)
- Try out different models for predicting the gross revenue of a movie (see Thinkful's Intro to Predictive Modeling workshops/webinars for additional support)
- Go back and impute some of the missing data that we dropped